In [ ]:
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10
import numpy as np
from torch.utils.data import DataLoader, Subset, Dataset
from tqdm import tqdm
import albumentations as A 
from albumentations.pytorch import ToTensorV2
import pickle
device = torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=3, padding=1),
            nn.BatchNorm2d(6),
            nn.GELU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.GELU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Linear(16*6*6, 120),
            nn.GELU(),
            nn.Dropout(p=0.5),
            nn.Linear(120, 84),
            nn.GELU(),
            nn.Dropout(p=0.5),
            nn.Linear(84, 10),
        )
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [3]:
class VGG(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            *[
                *[
                    nn.Conv2d(3, 64, kernel_size=3, padding=1),
                    nn.BatchNorm2d(64),
                    nn.GELU(),
                ],
                *[
                    nn.Conv2d(64, 64, kernel_size=3, padding=1),
                    nn.BatchNorm2d(64),
                    nn.GELU(),
                ],
                *[nn.MaxPool2d(kernel_size=2, stride=2)],
                *[
                    nn.Conv2d(64, 128, kernel_size=3, padding=1),
                    nn.BatchNorm2d(128),
                    nn.GELU(),
                ],
                *[
                    nn.Conv2d(128, 128, kernel_size=3, padding=1),
                    nn.BatchNorm2d(128),
                    nn.GELU(),
                ],
                *[nn.MaxPool2d(kernel_size=2, stride=2)],
                *[
                    nn.Conv2d(128, 256, kernel_size=3, padding=1),
                    nn.BatchNorm2d(256),
                    nn.GELU(),
                ],
                *[
                    nn.Conv2d(256, 256, kernel_size=3, padding=1),
                    nn.BatchNorm2d(256),
                    nn.GELU(),
                ],
                # *[
                #     nn.Conv2d(256, 256, kernel_size=3, padding=1),
                #     nn.BatchNorm2d(256),
                #     nn.GELU(),
                # ],
                *[nn.MaxPool2d(kernel_size=2, stride=2)],
                *[
                    nn.Conv2d(256, 512, kernel_size=3, padding=1),
                    nn.BatchNorm2d(512),
                    nn.GELU(),
                ],
                *[
                    nn.Conv2d(512, 512, kernel_size=3, padding=1),
                    nn.BatchNorm2d(512),
                    nn.GELU(),
                ],
                # *[
                #     nn.Conv2d(512, 512, kernel_size=3, padding=1),
                #     nn.BatchNorm2d(512),
                #     nn.GELU(),
                # ],
                *[nn.MaxPool2d(kernel_size=2, stride=2)],
                *[
                    nn.Conv2d(512, 512, kernel_size=3, padding=1),
                    nn.BatchNorm2d(512),
                    nn.GELU(),
                ],
                *[
                    nn.Conv2d(512, 512, kernel_size=3, padding=1),
                    nn.BatchNorm2d(512),
                    nn.GELU(),
                ],
                # *[
                #     nn.Conv2d(512, 512, kernel_size=3, padding=1),
                #     nn.BatchNorm2d(512),
                #     nn.GELU(),
                # ],
                *[nn.MaxPool2d(kernel_size=2, stride=2)],
            ]
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.GELU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.GELU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 10),
        )
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [4]:
class Bottleneck(nn.Module):
    def __init__(self, in_features, out_features, stride=1,downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(in_features, out_features, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_features)
        self.conv2 = nn.Conv2d(
            out_features,
            out_features,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False,
        )
        self.bn2 = nn.BatchNorm2d(out_features)
        self.conv3 = nn.Conv2d(
            out_features, out_features * 4, kernel_size=1, bias=False
        )
        self.bn3 = nn.BatchNorm2d(out_features * 4)
        self.gelu = nn.GELU()
        self.downsample = downsample

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.gelu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.gelu(out)

        out = self.conv3(out)
        out = self.bn3(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.gelu(out)
        return out


# [3, 4, 6, 3]


class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.gelu = nn.GELU()
        self.maxpool = nn.MaxPool2d(3, stride=2, padding=1)
        self.layer1 = nn.Sequential(
            Bottleneck(64, 64, downsample=nn.Sequential(
                nn.Conv2d(64, 64 * 4, kernel_size=1, stride=1),
                nn.BatchNorm2d(64 * 4)
            )),
            Bottleneck(64 * 4, 64),
            Bottleneck(64 * 4, 64),
        )
        self.layer2 = nn.Sequential(
            Bottleneck(64 * 4, 128, stride=2, downsample=nn.Sequential(
                nn.Conv2d(64 * 4, 128 * 4, kernel_size=1, stride=2),
                nn.BatchNorm2d(128 * 4)
            )),
            Bottleneck(128 * 4, 128),
            Bottleneck(128 * 4, 128),
            Bottleneck(128 * 4, 128),
        )
        self.layer3 = nn.Sequential(
            Bottleneck(128 * 4, 256, stride=2, downsample=nn.Sequential(
                nn.Conv2d(128 * 4, 256 * 4, kernel_size=1, stride=2),
                nn.BatchNorm2d(256 * 4)
            )),
            Bottleneck(256 * 4, 256),
            Bottleneck(256 * 4, 256),
            Bottleneck(256 * 4, 256),
            Bottleneck(256 * 4, 256),
            Bottleneck(256 * 4, 256),
        )
        self.layer4 = nn.Sequential(
            Bottleneck(256 * 4, 512, stride=2, downsample=nn.Sequential(
                nn.Conv2d(256 * 4, 512 * 4, kernel_size=1, stride=2),
                nn.BatchNorm2d(512 * 4)
            )),
            Bottleneck(512 * 4, 512),
            Bottleneck(512 * 4, 512),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(512 * 4, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.gelu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
class Model:
    def __init__(self, model, optimizer, criterion, lr_scheduler):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.lr_scheduler = lr_scheduler
        self.train_losses = []
        self.train_accs = []
        self.val_losses = []
        self.val_accs = []
        self.best_valid = float(0)
        if isinstance(self.model, LeNet):
            self.name = "LeNet"
        elif isinstance(self.model, VGG):
            self.name = "VGG"
        elif isinstance(self.model, ResNet):
            self.name = "ResNet"

    def optim_to_str(self):
        if isinstance(self.optimizer, torch.optim.SGD):
            if self.optimizer.param_groups[0]["nesterov"]:
                return "NAG"
            else:
                return "SGD"
        if isinstance(self.optimizer, torch.optim.Adadelta):
            return "Adadelta"
        if isinstance(self.optimizer, torch.optim.NAdam):
            return "NAdam"
        if isinstance(self.optimizer, torch.optim.RAdam):
            return "RAdam"
        if isinstance(self.optimizer, torch.optim.Adamax):
            return "Adamax"

    def __str__(self):
        
        return self.name + "_with_" + self.optim_to_str()

    def train_one_epoch(self, trainloader):
        self.model.train()
        print(f"Training {self}")
        train_running_loss = 0.0
        train_running_correct = 0
        counter = 0
        for i, data in tqdm(enumerate(trainloader), total=len(trainloader)):
            counter += 1
            image, labels = data
            image = image.to(device)
            labels = labels.to(device)
            self.optimizer.zero_grad()
            outputs = self.model(image)
            loss = self.criterion(outputs, labels)
            train_running_loss += loss.item()
            _, preds = torch.max(outputs.data, 1)
            train_running_correct += (preds == labels).sum().item()
            loss.backward()
            self.optimizer.step()
        self.train_losses.append(train_running_loss / counter)
        self.train_accs.append(100.0 * (train_running_correct / len(trainloader.dataset)))

    def validate(self, testloader):
        self.model.eval()
        print(f"Validation {self}")
        valid_running_loss = 0.0
        valid_running_correct = 0
        counter = 0
        with torch.no_grad():
            for i, data in tqdm(enumerate(testloader), total=len(testloader)):
                counter += 1

                image, labels = data
                image = image.to(device)
                labels = labels.to(device)
                outputs = self.model(image)
                loss = self.criterion(outputs, labels)
                valid_running_loss += loss.item()
                _, preds = torch.max(outputs.data, 1)
                valid_running_correct += (preds == labels).sum().item()
        self.val_losses.append(valid_running_loss / counter)
        self.val_accs.append(
            100.0 * (valid_running_correct / len(testloader.dataset))
        )
    def step(self,epoch, train_loader, val_loader):
        self.train_one_epoch(train_loader)
        self.validate(val_loader)
        if isinstance(self.lr_scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
            self.lr_scheduler.step(self.val_accs[-1])
        else:
            self.lr_scheduler.step()
        print(f"Training loss: {self.train_losses[-1]:.3f}, training acc: {self.train_accs[-1]:.3f}")
        print(f"Validation loss: {self.val_losses[-1]:.3f}, validation acc: {self.val_accs[-1]:.3f}")
        print(f'Lr: {self.lr_scheduler.get_last_lr()}')
        self.save_best(epoch)
        print('-'*50)
        
        

    def save_best(self, epoch):
        if self.val_accs[-1] > self.best_valid:
            self.best_valid = self.val_accs[-1]
            print(f"\nBEST VALIDATION: {self.best_valid}")
            print(f"\nSAVING BEST MODEL FOR EPOCH: {epoch+1}\n")
            torch.save(
                {
                    "epoch": epoch + 1,
                    "model_state_dict": self.model.state_dict(),
                },
                f"best_{self}.pth",
            )
    def save_model(self, epochs):
        torch.save({
                    'epoch': epochs,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'loss': self.criterion,
                    }, f"{self}.pth")
    def save_train_data(self, epochs):
        data = {'name': str(self),'epochs': epochs, 'train_losses': self.train_losses,
                'train_accs': self.train_accs, 'val_losses': self.val_losses,
                'val_accs': self.val_accs, 'best_valid': self.best_valid}
        with open(f'{self}_dat.pkl', 'wb') as f:
            pickle.dump(data, f)
            
        

In [6]:
models = []
patience = 3
cooldown = 3
threshold = 1e-3


model = LeNet()#.to(device)
optimizer = torch.optim.SGD(model.parameters(), momentum=0.9)
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", patience=patience, min_lr=1e-5, cooldown=cooldown, threshold=threshold
)
models.append(Model(model, optimizer, criterion, lr_scheduler))
model = LeNet()#.to(device)
optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, nesterov=True)
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", patience=patience, min_lr=1e-5, cooldown=cooldown, threshold=threshold
)
models.append(Model(model, optimizer, criterion, lr_scheduler))
model = LeNet()#.to(device)
optimizer = torch.optim.Adadelta(model.parameters())
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", patience=patience, min_lr=1e-3, cooldown=cooldown, threshold=threshold
)
models.append(Model(model, optimizer, criterion, lr_scheduler))
model = LeNet()#.to(device)
optimizer = torch.optim.NAdam(model.parameters(), decoupled_weight_decay=True, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", patience=patience, min_lr=1e-5, cooldown=cooldown, threshold=threshold
)
models.append(Model(model, optimizer, criterion, lr_scheduler))

model = VGG()#.to(device)
optimizer = torch.optim.SGD(model.parameters(), momentum=0.9)
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", patience=patience, min_lr=1e-5, cooldown=cooldown, threshold=threshold
)
models.append(Model(model, optimizer, criterion, lr_scheduler))
model = VGG()#.to(device)
optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, nesterov=True)
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", patience=patience, min_lr=1e-5, cooldown=cooldown, threshold=threshold
)
models.append(Model(model, optimizer, criterion, lr_scheduler))
model = VGG()#.to(device)
optimizer = torch.optim.Adadelta(model.parameters())
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", patience=patience, min_lr=1e-3, cooldown=cooldown, threshold=threshold
)
models.append(Model(model, optimizer, criterion, lr_scheduler))
model = VGG()#.to(device)
optimizer = torch.optim.NAdam(model.parameters(),lr=6e-4, decoupled_weight_decay=True, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", patience=patience, min_lr=1e-5, cooldown=cooldown, threshold=threshold
)
models.append(Model(model, optimizer, criterion, lr_scheduler))

model = ResNet()#.to(device)
optimizer = torch.optim.SGD(model.parameters(), momentum=0.9)
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", patience=patience, min_lr=1e-5, cooldown=cooldown, threshold=threshold
)
models.append(Model(model, optimizer, criterion, lr_scheduler))
model = ResNet()#.to(device)
optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, nesterov=True)
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", patience=patience, min_lr=1e-5, cooldown=cooldown, threshold=threshold
)
models.append(Model(model, optimizer, criterion, lr_scheduler))
model = ResNet()#.to(device)
optimizer = torch.optim.Adadelta(model.parameters())
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", patience=patience, min_lr=1e-3, cooldown=cooldown, threshold=threshold
)
models.append(Model(model, optimizer, criterion, lr_scheduler))
model = ResNet()#.to(device)
optimizer = torch.optim.NAdam(model.parameters(), decoupled_weight_decay=True, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", patience=patience, min_lr=1e-5, cooldown=cooldown, threshold=threshold
)
models.append(Model(model, optimizer, criterion, lr_scheduler))

model = ResNet()#.to(device)
optimizer = torch.optim.RAdam(model.parameters(), decoupled_weight_decay=True, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", patience=patience, min_lr=1e-5, cooldown=cooldown, threshold=threshold
)
models.append(Model(model, optimizer, criterion, lr_scheduler))
model = VGG()#.to(device)
optimizer = torch.optim.RAdam(model.parameters(), decoupled_weight_decay=True, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", patience=patience, min_lr=1e-5, cooldown=cooldown, threshold=threshold
)
models.append(Model(model, optimizer, criterion, lr_scheduler))
model = LeNet()#.to(device)
optimizer = torch.optim.RAdam(model.parameters(), decoupled_weight_decay=True, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", patience=patience, min_lr=1e-5, cooldown=cooldown, threshold=threshold
)
models.append(Model(model, optimizer, criterion, lr_scheduler))

model = ResNet()#.to(device)
optimizer = torch.optim.Adamax(model.parameters())
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", patience=patience, min_lr=1e-5, cooldown=cooldown, threshold=threshold
)
models.append(Model(model, optimizer, criterion, lr_scheduler))
model = VGG()#.to(device)
optimizer = torch.optim.Adamax(model.parameters(), lr=6e-4)
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", patience=patience, min_lr=1e-5, cooldown=cooldown, threshold=threshold
)
models.append(Model(model, optimizer, criterion, lr_scheduler))
model = LeNet()#.to(device)
optimizer = torch.optim.Adamax(model.parameters())
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", patience=patience, min_lr=1e-5, cooldown=cooldown, threshold=threshold
)
models.append(Model(model, optimizer, criterion, lr_scheduler))


In [7]:
def print_model_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / 1024**2
    print('model size: {:.3f}MB'.format(size_all_mb))

In [8]:
[print_model_size(model.model) for model in models[::4]]

model size: 0.310MB
model size: 492.110MB
model size: 89.972MB
model size: 89.972MB
model size: 492.110MB


[None, None, None, None, None]

In [9]:
def get_train_transforms():
    return A.Compose(
        [
            A.HorizontalFlip(),
            A.Blur(blur_limit=3, p=0.1),
            A.MotionBlur(blur_limit=3, p=0.1),
            A.MedianBlur(
                blur_limit=3,
                p=0.1,
            ),
            A.ToGray(
                p=0.3,
            ),
            A.RandomBrightnessContrast(
                p=0.3,
            ),
            A.RandomGamma(
                p=0.3,
            ),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=1),
            ToTensorV2(p=1.0),
        ]
    )


def get_test_transforms():
    return A.Compose(
        [
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=1),
            ToTensorV2(p=1),
            
        ]
    )

In [10]:
transforms = {
    "train": get_train_transforms(),
    "val": get_test_transforms(),
}

In [11]:
class AlbDataset(Dataset):
    def __init__(self, subdata, transforms):
        super().__init__()
        self.subdata = subdata
        self.transforms = transforms

    def __getitem__(self, index):
        img, target = self.subdata[index]
        if self.transforms:
            img = self.transforms(image=np.array(img).astype(np.float32) / 255)["image"]
        return img, target
    def __len__(self):
        return len(self.subdata)

In [12]:
ds = CIFAR10(
    "./data/train",
    train=True,
    download=True,
)
ds_test = CIFAR10(
    "./data/test",
    train=False,
    download=True,
)
ds_train = AlbDataset(Subset(ds, range(0, 40000)), transforms['train'])
ds_val = AlbDataset(Subset(ds, range(40000, 50000)), transforms['val'])
batch_size = 512
train_data = DataLoader(
    ds_train, batch_size, shuffle=True, drop_last=True,pin_memory=True, num_workers=4
)
# _ = DataLoader(ds_test, batch_size, shuffle=False,pin_memory=True, num_workers=4)
val_data = DataLoader(ds_val, batch_size, shuffle=False,pin_memory=True, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
epochs = 30
for model in models:
    print(f"[INFO]: {model}")
    model.model.to(device)
    for epoch in range(0, epochs):
        print(f"[INFO]: Epoch {epoch+1} of {epochs}")
        model.step(epoch, train_data, val_data)
    model.save_model(epoch)
    model.save_train_data(epoch)
    del model.model
    torch.cuda.empty_cache()


[INFO]: VGG_with_Adamax
[INFO]: Epoch 1 of 30
Training VGG_with_Adamax


100%|██████████| 78/78 [00:29<00:00,  2.65it/s]

Validation VGG_with_Adamax



100%|██████████| 20/20 [00:02<00:00,  6.70it/s]


Training loss: 3.130, training acc: 14.690
Validation loss: 2.080, validation acc: 20.290
Lr: [0.0006]

BEST VALIDATION: 20.29

SAVING BEST MODEL FOR EPOCH: 1

--------------------------------------------------
[INFO]: Epoch 2 of 30
Training VGG_with_Adamax


...

Training loss: 1.159, training acc: 59.570
Validation loss: 0.977, validation acc: 65.280
Lr: [0.002]
--------------------------------------------------
[INFO]: Epoch 30 of 30
Training LeNet_with_Adamax



100%|██████████| 78/78 [00:09<00:00,  8.07it/s]

Validation LeNet_with_Adamax



100%|██████████| 20/20 [00:02<00:00,  8.94it/s]

Training loss: 1.156, training acc: 59.828
Validation loss: 0.972, validation acc: 65.840
Lr: [0.002]

BEST VALIDATION: 65.84

SAVING BEST MODEL FOR EPOCH: 30

--------------------------------------------------
